In [1]:
# !pip install --upgrade dash

# !pip install jupyter_dash

import warnings 
warnings.filterwarnings("ignore")

In [2]:
#Import the libraries

import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input,Output
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

from jupyter_dash import JupyterDash
# If running in a Jupyter environment, no need to import JupyterDash
#from dash import Dash, html



In [3]:
#We load a dataset from Python library

data = datasets.load_diabetes()
df = pd.DataFrame(data.data, columns = data.feature_names)
# df.head() # to see if the dataset has been normalized/standardized

In [4]:
df["target"]=data.target

In [5]:
df["target"]

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64

In [6]:
df['target'] = data.target
#We prepare X and y

X = df.drop('target', axis = 1)
y = df['target']

In [7]:
#We will fit the data into regression model for the dashboard --> Lasso

lasso = linear_model.Lasso(alpha = 0.05)
lasso.fit(X,y)

Lasso(alpha=0.05)

In [8]:
#We make predictions

y_pred = lasso.predict(X)
r2 = r2_score(y, y_pred)

In [9]:
app = JupyterDash(__name__)
#app = Dash(__name__)
#app = Dash()  # Note: No __name__ parameter here

In [10]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Diabetes Regression Analysis Dashboard"),
    dcc.Graph(id='boxplot'),
    dcc.Graph(id='scatterplot'),
    dcc.Graph(id='linear-regression'),
    html.Div(id='mse')])

#We indicate the output and the input
@app.callback(
    Output('boxplot', 'figure'),
    Output('scatterplot', 'figure'),
    Output('linear-regression', 'figure'),
    Output('mse', 'children'),
    Input('boxplot', 'selectedData'))

#Definition and update of the graphs

def update_plots(selected_data):
    boxplot = px.box(df.melt(id_vars='target', var_name='key', value_name='value'), x='key', y='value')
    scatterplot = px.scatter_matrix(df, dimensions=data.feature_names, color='target')
    linear_regression = px.scatter(x=y, y=y_pred, labels={'x': 'True Target Values', 'y': 'Predicted Target Values'}, 
                                   title='Linear Regression')
    r2_text = f"R2 Score: {r2:.2f}"
    
    return boxplot, scatterplot, linear_regression, r2_text


app.run_server(mode='inline')